# Contoso Sales Analysis Assistant

The following notebook includes a version of [demo 3](./demo-3-contoso-sales-analysis.ipynb) which uses [Azure AI Agent Service](https://techcommunity.microsoft.com/blog/azure-ai-services-blog/introducing-azure-ai-agent-service/4298357) to build the Contoso Sales Assistant.

## Azure AI Agent Service
Azure AI Agent service builds upon Azure OpenAI Assistants API to provide an **enterprise-grade** solution for building scalable agents safely and securely. It integrates with an **extensive ecosystem of tools** to enable agents to ground their knowledge in real-time data (e.g. Azure AI Search, Sharepoint and Bing Search) and to **act on behalf of users** (e.g. Logic Apps and Azure Functions).
Azure AI Agent Service also allows a flexible model choice, going beyond the OpenAI collection. 

The service is accessible through the [Azure AI Foundry SDK](https://techcommunity.microsoft.com/blog/aiplatformblog/ignite-2024-announcing-the-azure-ai-foundry-sdk/4295862), which provides a simplified coding experience to build AI applications. 

> [!IMPORTANT]
> The Azure AI Agent Service is currently in **private preview**. This early stage of development means the product is actively evovling, with significant updates and improvements expected. Users should anticipate changes as we work towards refining features, enhancing functionality, and expanding capabilities. We welcome feedback and contributions during this phase to help shape the future of the product.
[Join the waitlist](https://nam.dcv.ms/nzy5CEG6Br) to get access to the private preview.

## Installation

Refer to the README.md file in this folder for installation instructions.

### Load parameters

In [ ]:
import os, time
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import FunctionTool, ToolSet, CodeInterpreterTool, RequiredFunctionToolCall, SubmitToolOutputsAction, ToolOutput
from azure.ai.projects.models import MessageTextContent, MessageImageFileContent, ThreadMessage, ThreadRun
from typing import Iterable

load_dotenv(".env")

DATA_FOLDER = "data/"
thread = None
agent = None

### Import Libraries and open the contoso-sales SQLite database

In [ ]:
import json
from datetime import datetime
from PIL import Image
from IPython.display import display
import sqlite3
import pandas as pd

con = sqlite3.connect("./database/contoso-sales.db")

### Get revenue by region

Called by the Assistant

In [ ]:
def get_revenue_by_region() -> pd.DataFrame:
    query = """
        SELECT  
            region AS Region,  
            SUM(number_of_orders) AS Orders,  
            SUM(revenue) AS Revenue,  
            SUM(discount) AS Discount,  
            SUM(shipping_cost) AS ShippingCost,
            SUM(revenue) - SUM(discount) - SUM(shipping_cost) AS NetRevenue  
        FROM 
            sales_data
        GROUP BY Region ORDER BY NetRevenue DESC
    """
    result = pd.read_sql_query(query, con)
    return json.dumps({"revenues_by_region": result.to_dict(orient="records")})

### Get sales by month

Called by the Assistant. Expects month and year as input.

In [ ]:
def get_sales_by_month(month: int, year: int) -> pd.DataFrame:
    query = """
        SELECT   
            year || '-' || CASE 
                WHEN month < 10 THEN '0' || month 
                ELSE month 
                END AS Month,
        SUM(revenue) AS Revenue,
        SUM(revenue) - SUM(discount) - SUM(shipping_cost) AS NetRevenue
        FROM sales_data 
        WHERE Month = ? and Year = ?
        GROUP BY Month, Year 
        ORDER BY Year, Month
    """
    result = pd.read_sql_query(query, con, params=[month, year])
    return json.dumps({"sales_by_month": result.to_dict(orient="records")})

### Create an AI Project client

In [ ]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)

### Define the Agent tools

The are three tools defined:

1. code_interpreter
1. `get_revenue_by_region`: This function returns the revenue by region from the SQLite database.
1. `get_sales_by_month`: This function returns the sales by month from the SQLite database.


In [ ]:
from typing import Any, Callable, Set
user_functions: Set[Callable[..., Any]] = {
    get_revenue_by_region,
    get_sales_by_month
}

In [ ]:
toolset = ToolSet()
functions = FunctionTool(functions=user_functions)
toolset.add(functions)
code_interpreter = CodeInterpreterTool()
toolset.add(code_interpreter)

### Set the Assistant instruction context

Sets the context for the conversation. The instructions are equivalent to setting the system message for an OpenAI chat completion.

In [ ]:
instructions = (
    "You are an advanced sales analysis assistant for Contoso. Your role is to be polite, professional, helpful, and friendly while assisting users with their sales data inquiries.",

    "You get all the sales data from this app using the functions provided. This data includes sales revenue categorized by region, product category, product type, and broken down by year and month.",

    "Here are some examples of the data structure:",
    "- Regions: Africa, Asia, Europe, America",
    "- Product Categories: Climbing gear, Camping equipment, Apparel, etc.",
    "- Product Types: Jackets, Hammocks, Wetsuits, Crampons, Shoes, etc.",
    "- Months: 2023-01, 2023-08, 2024-02, etc.",
    "- Revenue: Numeric values representing the sales revenue.",
    "- Discounts: Numeric values representing the discounts applied to the sales.",
    "- Shipping Costs: Numeric values representing the shipping costs.",
    "- Net Revenue: you can calculate user revenue, discount and shipping cost",

    "Your responsibilities include the following:",
    "- Analyze and provide insights based on the available sales data.",
    "- Generate visualizations that help illustrate the data trends.",
    "- If a question is not related to sales or is outside your scope, respond with 'I'm unable to assist with that. Please contact IT for more assistance.'",
    "- If the user requests help or types 'help,' provide a list of sample questions that you are equipped to answer.",
    "- If the user is angry or insulting, remain calm and professional. Respond with, 'I'm here to help you. Let's focus on your sales data inquiries. If you need further assistance, please contact IT for support.'",

    "Remember to maintain a professional and courteous tone throughout your interactions. Avoid sharing any sensitive or confidential information.",

    f"The current date and time is: {datetime.now().strftime('%x %X')}.",
)

### Process Function calling

Loops through the conversation and calls the appropriate function based on the user input.

In [ ]:
def call_functions(project_client: AIProjectClient, thread_id: str, run: ThreadRun) -> None:
    tool_calls = run.required_action.submit_tool_outputs.tool_calls
    if not tool_calls:
        print("No tool calls provided - cancelling run")
        project_client.agents.cancel_run(thread_id=thread_id, run_id=run.id)
        return

    tool_outputs = []
    for tool_call in tool_calls:
        if isinstance(tool_call, RequiredFunctionToolCall):
            try:
                print(f"Executing tool call: {tool_call}")
                output = functions.execute(tool_call)
                tool_outputs.append(
                    ToolOutput(
                        tool_call_id=tool_call.id,
                        output=output,
                    )
                )
            except Exception as e:
                print(f"Error executing tool_call {tool_call.id}: {e}")

    print(f"Tool outputs: {tool_outputs}")
    if tool_outputs:
        project_client.agents.submit_tool_outputs_to_run(
            thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs
        )
    print(f"Run finished with status: {run.status}")

### Format and display the Agent Messages for text and images

Utility functions to format and display the Agent messages.

In [ ]:
def format_messages(messages: Iterable[ThreadMessage]) -> None:
    last_msg = messages.get_last_message_by_sender("assistant")

    if last_msg:
        for content in last_msg.content:
            if isinstance(content, MessageTextContent):
                print(f"Last Message: {content.text.value}")

            if isinstance(content, MessageImageFileContent):
                    print(f"Image File ID: {content.image_file.file_id}")
                    file_name = f"{content.image_file.file_id}_image_file.png"
                    project_client.agents.save_file(file_id=content.image_file.file_id, file_name=file_name)
                    image = Image.open(file_name)
                    image = image.resize((image.width // 2, image.height // 2), Image.LANCZOS)
                    project_client.agents.delete_file(content.image_file.file_id)
                    display(image)

### Process the user messages

Loops through the conversation and calls the appropriate function based on the user input.

In [ ]:
def process_message(content: str) -> None:
    # Create message to thread
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content=content,
    )
    print(f"Created message, ID: {message.id}")

    run = project_client.agents.create_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
           call_functions(project_client, thread.id, run) 

    # Fetch and log all messages
    messages = project_client.agents.get_messages(thread_id=thread.id)
    format_messages(messages)


### Create an Agent Object

The Agent is responsible for managing the conversation with the user.

In [ ]:
api_deployment_name = os.getenv("OPENAI_GPT_DEPLOYMENT")

agent = project_client.agents.create_agent(
    model=api_deployment_name, name="my-assistant", instructions="\n".join(instructions), toolset=toolset
)
print(f"Created agent, ID: {agent.id}")

### Create a thread

Threads in the Agent Service are designed to be session-based.
Each thread is a conversation between the user and the assistant.

In [ ]:
thread = project_client.agents.create_thread()
print(f"Created thread, thread ID {thread.id}")

### Have a conversation with the Agent

In [ ]:
process_message("what was the total sales data for region? Display as a table.")

In [ ]:
process_message("Pie chart of sales by region. Use different colors for each region.")

In [ ]:
process_message("Calculate the worldwide sales revenue. Display as a table.")

In [ ]:
process_message("what were the sales for March in 2023? Display as a table.")

## Cleaning up

In [ ]:
project_client.agents.delete_agent(agent.id)
con.close()
print("Deleted agent")